# goal: build a crew for customer outreach

## notes and questions
- every tool needs to be provided with a bare minimum of these two attributes: `name`, `description`.
- custom tools can be defined using the `BaseTool` class from crewai

## main code

In [1]:
import warnings
warnings.filterwarnings('ignore')  # don't bother showing warnings

from crewai import Agent, Task, Crew

from IPython.display import Markdown

import os
from utils import get_openai_api_key, pretty_print_result, get_serper_api_key
openai_api_key = get_openai_api_key()
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [2]:
# agents

# sales rep
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)


# lead sales rep
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=True,
    verbose=True
)

In [3]:
# tools

from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

# initialise tools
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [4]:
# custom tool

from crewai_tools import BaseTool

class SentimentAnalysisTool(BaseTool):
    name: str="Sentiment Analysis Tool"
    description: str=("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # custom tool code goes in the _run() function
        # for the sake of this example, this tool won't do any actual sentiment analysis and will return "positive" for every text
        return "positive"


# initialise tool
sentiment_analysis_tool = SentimentAnalysisTool()

In [5]:
# tasks

# researching and profiling leads
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)


# personalised outreach to leads
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [6]:
# assembling the crew

crew = Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2,
	memory=True
)

In [9]:
inputs = {
    "lead_name": "Serviceplan Group",
    "industry": "Advertising and Marketing",
    "key_decision_maker": "Florian Haller",
    "position": "CEO",
    "milestone": "Agentic Services"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of Serviceplan Group, a company in the Advertising and Marketing sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To begin compiling a detailed profile of Serviceplan Group, I need to gather comprehensive information about the company's background, key decision-makers, recent business developments, and potential needs. I will start by conducting an internet search to find general information about Serviceplan Group.

Action: Search the internet
Action Input: {"search_query": "Serviceplan Group company pr

In [10]:
Markdown(result)

Below are the series of personalized email drafts aimed at Florian Haller, CEO of Serviceplan Group, tailored to highlight how CrewAI's solutions can support and align with Serviceplan's goals and achievements:

**Email 1: Introduction and Alignment with Serviceplan's Achievements**

Subject: Propel Serviceplan's Innovation and Growth with CrewAI’s Cutting-Edge Solutions

Dear Mr. Haller,

I hope this message finds you well. My name is [Your Name], and I am a Sales Representative at CrewAI, where we leverage advanced AI technologies to enhance business operations and drive strategic growth.

Firstly, congratulations on Serviceplan Group's remarkable achievements in the past year! Achieving an 11% growth in overall revenue and an impressive 19% internationally speaks volumes about your leadership and the robust strategy of your team. Moreover, the establishment of AI Labs underscores Serviceplan’s commitment to innovation and staying at the forefront of the digital transformation industry.

At CrewAI, we specialize in providing AI-driven solutions that streamline operations, enhance decision-making, and boost revenue growth—alignments that perfectly complement the ongoing initiatives at Serviceplan. Our tools are designed to integrate seamlessly with existing systems, empowering them to be more efficient and responsive to the dynamic market demands.

Given Serviceplan’s recent expansion and the setup of the AI Labs, I believe there is a tremendous opportunity for our organizations to collaborate. Our solutions could further enhance your capabilities, especially in predictive analytics and customer insights, areas where AI can dramatically increase accuracy and efficiency.

I would love the opportunity to discuss how CrewAI can specifically add value to Serviceplan Group. Are you available for a brief call next week? I am flexible with timings and can adjust to fit your schedule. We are keen to support Serviceplan Group in achieving your goals for this year and beyond.

Thank you for considering this potential synergy. I am looking forward to the possibility of working together to drive further success at Serviceplan Group.

Warm regards,

[Your Full Name]  
Sales Representative, CrewAI  
[Your Contact Information]  
[Your LinkedIn Profile] (Optional)

---

**Email 2: Enhancing Advertising Efficiency and Creativity with AI-Driven Strategies**

Subject: Elevate Your Creative Edge with CrewAI’s Innovative Solutions

Dear Mr. Haller,

I hope this message finds you well. Following our initial conversation, I’ve been inspired by Serviceplan Group’s ambitious vision, particularly your initiatives with AI Labs in Munich and San Francisco. At CrewAI, we share your passion for harnessing technology to break new ground in creativity and efficiency.

Our AI-driven marketing tools are designed to seamlessly integrate with and amplify the innovative work being done at your AI Labs. These tools not only enhance advertising creativity but also improve campaign efficiency, ensuring your messages resonate more deeply with diverse audiences across all markets.

I would love the opportunity to discuss how our solutions can specifically benefit Serviceplan Group. Could we schedule a brief call next week to explore this further? I am available at your convenience and eager to help drive your creative initiatives to new heights.

Thank you for considering this opportunity to innovate together.

Best regards,

[Your Name]  
Sales Representative | CrewAI  
[Your Contact Information]

---

**Email 3: Leveraging Data Analytics for Operational Efficiency and Market Insights**

Subject: Unlock New Market Potentials with Tailored Data Insights

Dear Mr. Haller,

I trust you found our last discussion on AI-driven marketing strategies insightful. Building on that, I would like to delve deeper into how CrewAI’s advanced data analytics can transform Serviceplan Group’s operational efficiency and enhance customer insights, particularly in burgeoning markets like Italy.

Our analytics platforms are adept at parsing complex market data to deliver actionable insights, enabling companies like yours to make informed decisions swiftly and with greater accuracy. This capability is crucial for effectively navigating and expanding in new international markets.

Could we arrange a deeper discussion or perhaps a demo presentation to show you firsthand how our tools can be integrated into your current operations? Please let me know a time that works best for you, and I will make the necessary arrangements.

Looking forward to our continued collaboration.

Warm regards,

[Your Name]  
Sales Representative | CrewAI  
[Your Contact Information]